In [1]:
import pandas as pd
import pandasql as ps
import numpy as np
import matplotlib.pyplot as plt

An online retail store has hired you as a consultant to review their data and provide insights that would be valuable to the CEO and CMO of the business. The business has been performing well and the management wants to analyse what the major contributing factors are to the revenue so they can strategically plan for next year.

# 0. Dataset

In [2]:
tata_data = pd.read_excel('online_retail.xlsx')

In [3]:
tata_data['StockCode'] = tata_data['StockCode'].astype(str)
tata_data['TotalPrice'] = tata_data['Quantity'] * tata_data['UnitPrice']

In [5]:
print(tata_data.shape)
tata_data.head(3)

(541909, 11)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,InvoiceYear,InvoiceMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom,15.30,2010,12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom,20.34,2010,12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom,22.00,2010,12


# 1. Questions of interest to the CEO

## 1.1. Which region is generating the highest revenue, and which region is generating the lowest?

In [6]:
ps.sqldf('''SELECT 
            Country, 
            SUM(TotalPrice) AS TotalSum
         FROM
            tata_data
         GROUP BY
            Country
         ORDER BY
            TotalSum DESC LIMIT 5''')

,Country,TotalSum
0,United Kingdom,8.187806e+06
1,Netherlands,2.846615e+05
2,EIRE,2.632768e+05
3,Germany,2.216982e+05
4,France,1.974039e+05


In [7]:
ps.sqldf('''SELECT 
            Country, 
            SUM(TotalPrice) AS TotalSum
         FROM
            tata_data
         GROUP BY
            Country
         ORDER BY
            TotalSum ASC LIMIT 5''')

,Country,TotalSum
0,Saudi Arabia,131.17
1,Bahrain,548.40
2,Czech Republic,707.72
3,RSA,1002.31
4,Brazil,1143.60


## 1.2. What is the monthly trend of revenue, which months have faced the biggest increase/decrease?,

In [8]:
ps.sqldf('''SELECT
    InvoiceMonth,
    InvoiceYear,
    ROUND(MonthTotalPrice,2) as MonthTotalLrice,
    ROUND(100 * (1 - LAG(MonthTotalPrice) OVER (ORDER BY InvoiceYear, InvoiceMonth) / MonthTotalPrice), 2) as PercentageChanging
FROM
(SELECT
    InvoiceMonth,
    InvoiceYear,
    SUM (TotalPrice) as MonthTotalPrice
FROM
    tata_data
GROUP BY
    InvoiceMonth, InvoiceYear
ORDER BY
    InvoiceYear ASC,
    InvoiceMonth ASC) as TataGroupMonth''')

,InvoiceMonth,InvoiceYear,MonthTotalLrice,PercentageChanging
0,12,2010,748957.02,NaN
1,1,2011,560000.26,-33.74
2,2,2011,498062.65,-12.44
3,3,2011,683267.08,27.11
4,4,2011,493207.12,-38.54
5,5,2011,723333.51,31.81
6,6,2011,691123.12,-4.66
7,7,2011,681300.11,-1.44
8,8,2011,682680.51,0.20
9,9,2011,1019687.62,33.05


## 1.3. Which months generated the most revenue? Is there a seasonality in sales?

In [9]:
ps.sqldf('''SELECT
    InvoiceMonth,
    InvoiceYear,
    ROUND(SUM (TotalPrice), 2) as MonthTotalPrice
FROM
    tata_data
GROUP BY
    InvoiceMonth, InvoiceYear
ORDER BY
    3 DESC LIMIT 5''')

,InvoiceMonth,InvoiceYear,MonthTotalPrice
0,11,2011,1461756.25
1,10,2011,1070704.67
2,9,2011,1019687.62
3,12,2010,748957.02
4,5,2011,723333.51


In [10]:
ps.sqldf('''SELECT
    InvoiceMonth,
    InvoiceYear,
    ROUND(SUM (TotalPrice), 2) as MonthTotalPrice
FROM
    tata_data
GROUP BY
    InvoiceMonth, InvoiceYear
ORDER BY
    3 ASC LIMIT 5''')

,InvoiceMonth,InvoiceYear,MonthTotalPrice
0,12,2011,433668.01
1,4,2011,493207.12
2,2,2011,498062.65
3,1,2011,560000.26
4,7,2011,681300.11


## 1.4. Who are the top customers and how much do they contribute to the total revenue? Is the business dependent on these customers or is the customer base diversified?

In [11]:
ps.sqldf('''SELECT
    CustomerID,
    MaxCustomerPrice.Country,
    ROUND (MaxCustomerIDCountry / TotalSum * 100, 2) AS Percentage
FROM
   (SELECT
        CustomerID,
        Country,
        MAX(CustomerIDSum) AS MaxCustomerIDCountry
    FROM
   (SELECT
        CustomerID,
        Country,
        ROUND (SUM(TotalPrice), 2) AS CustomerIDSum
    FROM
        tata_data
    WHERE
        CustomerID IS NOT NULL
    GROUP BY
        CustomerID) AS CustomerPrice
GROUP BY
    Country) AS MaxCustomerPrice
JOIN
   (SELECT
    Country,
    ROUND (SUM (TotalPrice), 2) as TotalSum
    FROM
        tata_data 
    GROUP BY
        Country) AS CountryTotalSum
ON
    MaxCustomerPrice.Country = CountryTotalSum.Country
ORDER BY
    3 DESC''')

,CustomerID,Country,Percentage
0,12769.0,Brazil,100.00
1,12781.0,Czech Republic,100.00
2,15108.0,European Community,100.00
3,12347.0,Iceland,100.00
4,12764.0,Lebanon,100.00
5,15332.0,Lithuania,100.00
6,12446.0,RSA,100.00
7,12565.0,Saudi Arabia,100.00
8,12744.0,Singapore,100.00
9,14646.0,Netherlands,98.18


# 2. Questions of interest to the CMO

## 2.1. What is the percentage of customers who are repeating their orders?

In [12]:
customers_rep = ps.sqldf('''
         SELECT
            CustomerID,
            Country,
            COUNT(InvoiceNo) as Orders
         FROM
         (SELECT
            InvoiceNo,
            CustomerID,
            Country,
            COUNT(StockCode)
         FROM
            tata_data
         WHERE
            CustomerID IS NOT NULL
         GROUP BY
            InvoiceNo) AS InvoiceNO
         GROUP BY
            CustomerID
         HAVING
            COUNT(InvoiceNo) > 1
         ''')
customers_total = ps.sqldf('''
         SELECT
            CustomerID,
            Country,
            COUNT(InvoiceNo) as Orders
         FROM
         (SELECT
            InvoiceNo,
            CustomerID,
            Country,
            COUNT(StockCode)
         FROM
            tata_data
         WHERE
            CustomerID IS NOT NULL
         GROUP BY
            InvoiceNo) AS InvoiceNO
         GROUP BY
            CustomerID
         ''')

In [13]:
customers_rep.shape[0] / customers_total.shape[0]

0.69967978042086

## 2.2. For the repeat customers, how long does it take for them to place the next order after being delivered the previous one?

In [17]:
orders = ps.sqldf('''
         SELECT
            InvoiceNo,
            CustomerID,
            COUNT(InvoiceNo) OVER(PARTITION BY CustomerID) AS Orders,
            InvoiceDate,
            LAG(InvoiceDate) OVER(PARTITION BY CustomerID) AS PrevInvoiceDate
         FROM
         (SELECT
            InvoiceNo,
            InvoiceDate,
            CustomerID,
            COUNT(StockCode)
         FROM
            tata_data
         WHERE
            CustomerID IS NOT NULL
         GROUP BY
            InvoiceNo) AS Repeated
         ''')
orders['DiffInvoiceDate'] = abs(pd.to_datetime(orders['InvoiceDate']) - pd.to_datetime(orders['PrevInvoiceDate'])).dt.days

In [18]:
ps.sqldf('''
SELECT
   CustomerID,
   ROUND(AVG(DiffInvoiceDate), 1) AS AvgData
FROM
   orders
GROUP BY
   CustomerID
HAVING
   AVG(DiffInvoiceDate) IS NOT NULL
''')

,CustomerID,AvgData
0,12346.0,0.0
1,12347.0,60.8
2,12348.0,94.3
3,12352.0,52.8
4,12356.0,151.5
...,...,...
3054,18276.0,11.0
3055,18277.0,260.0
3056,18282.0,117.0
3057,18283.0,22.3


## 2.3. What revenue is being generated from the customers who have ordered more than once?

In [48]:
orders_customers = ps.sqldf('''
         SELECT
            CustomerId,
            Country,
            ROUND(COUNT(InvoiceNo), 2) AS Orders,
            SUM(PriceStockCode) AS TotalSum
         FROM
         (SELECT
            InvoiceNo,
            CustomerID,
            Country,
            COUNT(StockCode),
            SUM(TotalPrice) as PriceStockCode
         FROM
            tata_data
         WHERE
            CustomerID IS NOT NULL
         GROUP BY
            InvoiceNo) AS CountStock
         GROUP BY
            CustomerID
         HAVING
            COUNT(InvoiceNo) > 1
         ''')
ps.sqldf('''
         SELECT
            CountryOrderSum.Country AS Country,
            Customers,
            ROUND((OrdersSum / TotalSum) * 100, 2) AS Percentage
         FROM
         (SELECT
            Country,
            COUNT(CustomerID) AS Customers,
            SUM(TotalSum) AS OrdersSum
         FROM
            orders_customers
         GROUP BY
            Country) AS CountryOrderSum
         JOIN
            (SELECT
               Country,
               ROUND (SUM (TotalPrice), 2) as TotalSum
            FROM
               tata_data 
            GROUP BY
               Country) AS CountryTotalSum
            ON
               CountryOrderSum.Country = CountryTotalSum.Country
         ''')

,Country,Customers,Percentage
0,Australia,9,101.01
1,Austria,5,73.45
2,Belgium,18,95.77
3,Canada,1,80.19
4,Channel Islands,6,82.18
5,Cyprus,4,97.16
6,Czech Republic,1,100.00
7,Denmark,7,96.68
8,EIRE,3,95.07
9,European Community,1,100.00


In Australia there were orders with negative values and customers with orders with negative sums

## 2.4. Who are the customers that have repeated the most? How much are they contributing to revenue?

In [30]:
ps.sqldf('''
         SELECT
            CustomerID,
            Country,
            COUNT(InvoiceNo) as Orders
         FROM
         (SELECT
            InvoiceNo,
            CustomerID,
            Country,
            COUNT(StockCode)
         FROM
            tata_data
         WHERE
            CustomerID IS NOT NULL
         GROUP BY
            InvoiceNo) AS InvoiceNO
         GROUP BY
            CustomerID
         HAVING
            COUNT(InvoiceNo) > 50
         ''')

,CustomerID,Country,Orders
0,12748.0,United Kingdom,224
1,12971.0,United Kingdom,89
2,13089.0,United Kingdom,118
3,13408.0,United Kingdom,81
4,13694.0,United Kingdom,60
5,13767.0,United Kingdom,52
6,13798.0,United Kingdom,63
7,14156.0,EIRE,66
8,14527.0,United Kingdom,86
9,14606.0,United Kingdom,128
